In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
warnings.filterwarnings('ignore')
import regex as re
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit    
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

import time
today = time.strftime("%Y_%m_%d")
print(today)

# parameters
customer_id = '5'
formatted_attribute = 'Weight'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=767
attribute_id_input=2674

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['LONG_DESCRIPTION']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

df.drop_duplicates(subset='external_id', inplace=True)
df=df[(df['external_id'].astype(str)!='65287538')&(df['external_id'].astype(str)!='69486862')&(df['external_id'].astype(str)!='200045714')]
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

2023_04_25
16501
[]


In [2]:
regex_pattern = r"(?i)((?<!support.{0,10})(?<!Minimum.{0,10}?)(?<!Maxi?m?u?m?\s?c?h?i?l?d?.{0,10}?)(?<!accommodates.{10})(?<!light)weigh(?!ts,)(?!t,)(?!t\s?capacity)(?!ing).{0,11}(?<!lb.{0,10})lb\s?[1-9]{0,3})|((?<=net.weight.)\d+(?:\.\d+)?[^\d]?lb)|()"               
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

df['matches']=df['matches'].apply(lambda x: natsorted(x))
df_one=df[df['matches'].astype(str)!='[]']
print(len(df_one))

# df_one['matchez']=df_one['matches'].apply(lambda x: re.sub(r'''(?i)(?<=\d)\\?"? lb\]',' lb"]''',str(x)))
df_one['matchez']=df_one['matches'].apply(lambda x: natsorted(x)).apply(lambda x: re.sub(r'(?i)(\s?just\s?)|(weight?s?\s?(?:total|of|under|\=|about|is)\s?)|(weight of )|(only)|(up.?to)|(\\")|(limit)|(\,\'?\s?\'?weight or \d+\s?lb)|(weight(?:s|ed)?(?:\s?at\s?)?)|(\(each\))|(within)|()','',str(x))).apply(lambda x: re.sub(r'''(?i)(weighs?t?\s?o?f?\s?)|(\s?weight\s)|((?<=\d)\s(?=[A-z]))|(\[)|(\])|(')|(\\)|(xa0\s?)|(eigs\s?)|\(|\)|(\\?"\s?lb)''','',str(x))).apply(lambda x: re.sub(r"-",' ',str(x))).apply(lambda x: re.sub(r'(?i)(?<=\d)lb|\s?(?<!l)b|\s?lb|" lb',' lb',str(x))).apply(lambda x: re.sub(r'(?i)(?<=\d)"?\]','lb"]',str(x))).apply(lambda x: re.sub(r'((?<=\.)\s(?=\d))|()','',str(x)))                                   
curate(df_one,'matchez',formatted_attribute)

print(len(df_one))
rounding_lbs(df_one,curation_col,'ac-km-z') 
df_one=df_one[df_one['rounding'].astype(str)=='[]']
print(len(df_one))

# df_one[curation_col]=df_one[curation_col].apply(lambda x: re.sub(r'(?i)(?<=\d)lb|\s?(?<!l)b|\s?lb',' lb',str(x)))
# # df_one[curation_col]=df_one[curation_col].apply(lambda x: re.sub(r'\[','["',str(x))).apply(lambda x: re.sub(r'\]','"]',str(x))).apply(lambda x: re.sub(r"'",'',str(x))).apply(lambda x: re.sub(r",",'","',str(x))).apply(lambda x: re.sub(r'" |""','"',str(x))).apply(lambda x: re.sub(r'(?<=\d)lb',' lb',str(x)))
# df_one[curation_col]=df_one[curation_col].apply(lambda x: re.sub(r'(?i)(w?e?i?g?h?s?t?)','',str(x))).apply(lambda x: re.sub(r'(?i)((?<!l)b)','lb',str(x))).apply(lambda x: re.sub(r'(?i)(lb)','lb',str(x))).apply(lambda x: re.sub(r'(?i)xa0\s?|eigs\s?|\(|\)','',str(x))).apply(lambda x: re.sub(r'"\s','"',str(x))).apply(lambda x: re.sub('\s?"\s?\,\s?"\s?','","',str(x)))                           
matchesnewest=df_one[['external_id',curation_col]]
matchesnewest[curation_col]=matchesnewest[curation_col].apply(lambda x: re.sub(r'""','"',str(x))).apply(lambda x: re.sub(r'(?i)(?<!l)b','lb',str(x))).apply(lambda x: re.sub(r'(?<=\d)lb',' lb',str(x)))


751
Repeated External IDs: []
751
612


In [3]:
regex_pattern_na=r'(?i)((?:supports?.{0,30}|rated|store|hold)?.?up.?to.?\d+.?lb)|(suitable.{0,25}\d+.?lb)|(support.{0,15}weighi?n?g?.?\d.?lb)|(support.{0,15}\d+.?lb)|((?<!weighs.)(?<!)\d+\,?\.?\d*.?lb.{0,7}(?:static.?load.?|max|seating|load|distributed|pound|maxi?m?u?m?)?.?(?:weight)?.?capacity)|(users?.?up.?to.?\d+.?lb)|(\d+.?lb.{0,7}break.?strength)|((?:weight|load).?capacity.?o?f?.{0,5}\d+.?(?:lb)?)|((?<!weighs.)\s\d+\s?lb.?\s?weight capacity)|([^weighs][^\)\.]\s\d+\s?lb.?\s?weight capacity)|([^weighs].?\d+\,?\.?\d*.?\s?lb.?\s?(?:maxi?m?u?m?)?.?(?:weight|load).?capacity)|(weight.?capacity.{0,5}(?:seating)?.{0,}\d+.?lb)|((?:maxi?m?u?m?|mini?m?u?m?).{0,15}(?:weight)?.?\d+.?lb)|(seats.{0,15}\d+.?lb)|(product.?weight)'     
df['matches_na']=df['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))
df_na=df[(df['matches'].astype(str)=='[]')]#&(df['matches_na'].astype(str)=='[]')]
print(len(df_na))
df_na[curation_col]='n/a'
match_na=df_na[['external_id',curation_col]]

15750


In [4]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(match_na))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnewest))

match_na
matchesnewest


In [5]:
print(len(df))
print(len(match_na)+len(matchesnewest))

16501
16362


In [6]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [7]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute,match_na,today)

In [8]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute,matchesnewest,today)